# Installation and Model Loading

In [ ]:
%pip install unsloth
%pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install transformers datasets trl

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None      # Automatically choose optimal data type (e.g. float16/bfloat16)
load_in_4bit = True  # Enable 4-bit quantization for memory efficiency

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
    # token= HF_TOKEN
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config = None
)


In [ ]:
import json

# ds_prompt = """
# A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
# The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
# The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively,
# i.e., <think> reasoning process here </think> <answer> answer here </answer>.

# User: Analyze the sentiment and aspects in the following review text and output the labels in a json containing "aspect", "opinion", "polarity":
# Review: {review}

# Assistant:
# <think>
# Step 1: Parse the input text and extract the sentiment elements.
# Step 2: Identify aspects, opinions, polarity, and categories based on the text.
# Step 3: Structure the output as a JSON object with "labels" keys, matching the ABSA dataset format exactly.
# </think>
# <answer>
# {answer}
# </answer>
# """

ds_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before giving your final answer, carefully work through a step-by-step chain-of-thought that details your reasoning process.

### Instruction:
You are an expert in aspect-based sentiment analysis (ABSA) with a deep understanding of laptop reviews. Your task is to extract key aspects and their corresponding sentiments from the review text. Follow these guidelines precisely:
1. **Aspect Extraction**: Identify aspects by extracting only the main nouns that denote features or components mentioned in the review. Do not include any adjectives, adverbs, or additional modifiers. The extracted aspect must appear verbatim in the review text.
2. **Sentiment Determination**: For each extracted aspect, determine the sentiment expressed in the review—either "positive", "negative", or "neutral".
3. **Output Format**: Provide your final answer in JSON format with two keys: "aspects" (a list of the identified aspect nouns) and "sentiments" (a list of the corresponding sentiment labels).
4. **Chain-of-Thought Requirement**: Before giving the final JSON output, include a detailed chain-of-thought explaining each step of your reasoning. This explanation should be enclosed within `<chain_of_thought></chain_of_thought>` and the final answer should be enclosed within `<final_answer></final_answer>`.

### Review:
{review}

### Response:
<chain_of_thought>
Step 1: Read and understand the review text.
Step 2: Identify phrases that express opinions about laptop features.
Step 3: For each opinion phrase, isolate the core noun (aspect) that exists verbatim in the review text, and strictly **NO ADJECTIVES**, if the word is not a noun, **ignore it** and **dont convert to a noun**.
Step 4: Determine the sentiment for each identified aspect (all sentiments in this example are positive), double check if the aspect is indeed a noun, if it is not, dont include the corresponding aspect and sentiment in the output
Step 5: Prepare the final JSON output as instructed.
</chain_of_thought>
<final_answer>
{answer}
</final_answer>
"""

EOS_TOKEN = tokenizer.eos_token
def format_absa_example(example, tokenizer):
    # Extract the review text.
    review = example["text"]
    labels = example["aspects"]
    # # Process each label.
    # # Replace "NULL" aspects with "general" (or another default if desired).
    # formatted_labels = []
    # for label in example["labels"]:
    #     aspect = label["aspect"] if label["aspect"] != "NULL" else "general"
    #     opinion = label["opinion"]
    #     polarity = label["polarity"]
    #     category = label["category"]
    #     formatted_labels.append(f"{aspect}: {opinion} ({polarity}, {category})")

    # # Join all formatted labels with a separator.
    # expected_output = "; ".join(formatted_labels)

    # Construct the prompt.

    prompt = ds_prompt.format(review = review, answer = f"'aspects': {labels['term']} 'sentiment': {labels['polarity']}")

    # Combine prompt and expected output, appending the end-of-sequence token.
    # full_text = prompt + " " + expected_output + tokenizer.eos_token
    full_text = prompt + EOS_TOKEN
    return {"text": full_text}


In [ ]:
example1 = {'id': '2005',
 'text': 'it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous.',
 'aspects': {'term': ['quality', 'GUI', 'applications', 'use'],
  'polarity': ['positive', 'positive', 'positive', 'positive'],
  'from': [14, 36, 118, 143],
  'to': [21, 39, 130, 146]}}

In [ ]:
print(format_absa_example(example1, tokenizer))

# Model inference before fine-tuning

In [ ]:
question = "it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous."


FastLanguageModel.for_inference(model) 
inputs = tokenizer([ds_prompt.format(review = question, answer = "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

#"aspects": [ "quality", "GUI", "applications", "use" ], "sentiments": [ "positive", "positive", "positive", "positive" ]

In [ ]:
from datasets import load_dataset


# ds_train = load_dataset("json", data_files="laptop_quad_train.tsv.jsonl")
# ds_test = load_dataset("json", data_files="laptop_quad_test.tsv.jsonl")
ds_train = load_dataset("jakartaresearch/semeval-absa", "laptop", split = "train")
# Apply formatting (using batched map if desired)
# ds = ds.map(lambda examples: format_absa_example(examples, tokenizer), batched=False)
ds_train[3]

In [ ]:
ds_train = ds_train.map(lambda examples: format_absa_example(examples, tokenizer), batched=False)

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=200,  # Increase steps as needed for your dataset size
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs_absa",
    report_to="none"  # or "wandb" if you use Weights & Biases
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds_train,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # adjust as needed
    args=training_args,
)


In [ ]:
trainer_stats = trainer.train()

# Model inference after fine-tuning

In [ ]:
 # For inference, set the model to inference mode
FastLanguageModel.for_inference(model)

# Example input – you can re-use the prompt format
question = "it is of high quality, has a killer GUI, is extremely stable, is highly expandable, is bundled with lots of very good applications, is easy to use, and is absolutely gorgeous."
inputs = tokenizer([ds_prompt.format(review = question, answer = "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=200,
    use_cache=True,
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
ds_train['validation'][501]

In [ ]:
model.save_pretrained("DeepSeek-R1-ABSA")
tokenizer.save_pretrained("DeepSeek-R1-ABSA")